# Here is work on categories

In [12]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from product.models import Category, CategoryTags
from django.db import transaction
from django.db.models import Q
import pandas as pd
import pickle


import MySQLdb

db = MySQLdb.connect('localhost', 'root', 'manhee33338', 'angara_interface',  cursorclass=MySQLdb.cursors.DictCursor)
#db.set_character_set('utf8')
cursor = db.cursor()

# table = 'group_dict_groups'
table = 'product_category'

df = pd.read_sql('SELECT id,name FROM ' + table + ' WHERE id > 1999', con=db)

In [7]:
# -*- coding: utf-8 -*-
# Портирован с Java по мотивам http://www.algorithmist.ru/2010/12/porter-stemmer-russian.html
import re

class Porter:
	PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
	REFLEXIVE = re.compile(u"(с[яь])$")
	ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
	PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
	VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
	NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
	RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
	DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
	DER = re.compile(u"ость?$")
	SUPERLATIVE = re.compile(u"(ейше|ейш)$")
	I = re.compile(u"и$")
	P = re.compile(u"ь$")
	NN = re.compile(u"нн$")

	def stem(word):
		word = word.lower()
		word = word.replace(u'ё', u'е')
		m = re.match(Porter.RVRE, word)
		if m and m.groups():
			pre = m.group(1)
			rv = m.group(2)
			temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
			if temp == rv:
				rv = Porter.REFLEXIVE.sub('', rv, 1)
				temp = Porter.ADJECTIVE.sub('', rv, 1)
				if temp != rv:
					rv = temp
					rv = Porter.PARTICIPLE.sub('', rv, 1)
				else:
					temp = Porter.VERB.sub('', rv, 1)
					if temp == rv:
						rv = Porter.NOUN.sub('', rv, 1)
					else:
						rv = temp
			else:
				rv = temp
			
			rv = Porter.I.sub('', rv, 1)

			if re.match(Porter.DERIVATIONAL, rv):
				rv = Porter.DER.sub('', rv, 1)

			temp = Porter.P.sub('', rv, 1)
			if temp == rv:
				rv = Porter.SUPERLATIVE.sub('', rv, 1)
				rv = Porter.NN.sub(u'н', rv, 1)
			else:
				rv = temp
			word = pre+rv
		return word
	stem=staticmethod(stem)

if __name__ == '__main__':
	print (Porter.stem(u'устойчивость'))

устойчив


# Code for tagging categories almost until end

In [8]:
s = Porter()
print(s.stem('двери'))

двер


In [9]:
import pprint as p

In [10]:
unique_w_list = []
for index, row in df.iterrows():
#     print(row['id'], row['name'])
    words = row['name'].split(' ')
    for word in words:
        word = s.stem(word)
        if word not in unique_w_list and len(word) > 3:
            unique_w_list.append(word)
print(len(unique_w_list))

495


In [16]:
unique_w_list[:10]

['радиатор',
 'печк',
 'маслян',
 'колодк',
 'тормозн',
 'бампер',
 'передн',
 'задн',
 'абсорбер',
 'креплен']

In [20]:
CategoryTags.objects.all().delete()

(495, {'product.Category_tags': 0, 'product.CategoryTags': 495})

In [21]:
for tag in unique_w_list:
    new_tag = CategoryTags.objects.create(
    name = tag
    )

In [22]:
categories = Category.objects.all()

In [23]:
final_dict = {}
for n in unique_w_list:
    gr_list = []
    for index, group in df.iterrows():
        if n in group['name']:
            gr_list.append({'id': group['id'], 'name': group['name']})
    final_dict[n] = gr_list
p.pprint(final_dict)

{'абсолютн': [{'id': 2251, 'name': 'датчик абсолютного давления'}],
 'абсорбер': [{'id': 2007, 'name': 'бампера абсорбер'},
              {'id': 2433, 'name': 'клапан абсорбера'}],
 'аварийк': [{'id': 2393, 'name': 'выключатель аварийки'},
             {'id': 2395, 'name': 'кнопка аварийки'}],
 'автомагнитол': [{'id': 3353, 'name': 'автомагнитола'}],
 'автоматическ': [{'id': 2178, 'name': 'дверь автоматическая'}],
 'автономк': [{'id': 2887, 'name': 'предохранитель автономки'}],
 'аккумулятор': [{'id': 2087, 'name': 'аккумулятор'},
                 {'id': 2088, 'name': 'аккумулятор клемма'},
                 {'id': 2089, 'name': 'аккумулятор крепление'},
                 {'id': 2090, 'name': 'аккумулятор крышка'},
                 {'id': 2091, 'name': 'аккумулятор предохранитель'},
                 {'id': 2092, 'name': 'аккумулятора кронштейн'},
                 {'id': 2093, 'name': 'аккумулятора провод'},
                 {'id': 2094, 'name': 'аккумулятора ящик'},
                 {'id

 'подкрылок': [{'id': 3143, 'name': 'подкрылок'}],
 'подлокотник': [{'id': 2641, 'name': 'крышка подлокотника'},
                 {'id': 3040, 'name': 'крепление подлокотника'},
                 {'id': 3157, 'name': 'подлокотник'},
                 {'id': 3158, 'name': 'подлокотник оригинальный'},
                 {'id': 3159, 'name': 'подлокотник штатный'},
                 {'id': 3160, 'name': 'подлокотник и консоль'},
                 {'id': 3161, 'name': 'подлокотник двери'}],
 'подножк': [{'id': 2017, 'name': 'бампер подножка'},
             {'id': 3239, 'name': 'подножка порога'}],
 'подогр': [{'id': 2204, 'name': 'зеркало с подогревом'},
            {'id': 2205, 'name': 'зеркало без подогрева'},
            {'id': 2384, 'name': 'блок управления подогревом сидений'},
            {'id': 2401, 'name': 'кнопка подогрева сидений'},
            {'id': 2886, 'name': 'предохранитель подогрева сидений'},
            {'id': 3222, 'name': 'подогрев зеркал'},
            {'id': 3223, 'name'

In [24]:
with open('tagged_categories.pickle', 'wb') as handle:
    pickle.dump(final_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
with open('tagged_categories.pickle', 'rb') as handle:
    final_dict = pickle.load(handle)

In [ ]:
for k, f in final_dict.items():
    tag = k
    for ctg in f:
        cat = Category.objects.get(id=ctg['id'])
        tg = CategoryTags.objects.get(name=tag)
        cat.tags.add(tg)
        print(ctg, tag)

In [28]:
category = Category.objects.all()
tag_cats = []
for cat in category:
    for tag in cat.tags.all():
        tag_cats.append([tag.name, cat.id, cat.name])

In [31]:
import csv
with open('tags.csv', 'w') as file:
    writer = csv.writer(file, delimiter=';')
    for line in tag_cats:
        writer.writerow(line)

In [ ]:
import pickle

with open('tagged_categories.pickle', 'wb') as handle:
    pickle.dump(final_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('tagged_categories.pickle', 'rb') as handle:
#     b = pickle.load(handle)

In [ ]:
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df.name)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.id
data_dtm

In [ ]:
data = data_dtm.transpose()

In [ ]:
data.head()

In [ ]:
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False)
    top_dict[c]= list(zip(top.index, top.values))

top_dict